# CDF Seismic API - Demo

This notebook is based on the Cognite Seismic Python SDK. The SDK itself is built on the Cognite CDF APIs.



### Install Cognite SDK and libraries
First we install dependencies and import a number of helpful Python modules, including the Cognite SDK itself:


In [ ]:
!pip install cognite-seismic-sdk shapely matplotlib geojson ipyleaflet >pipout && echo Dependencies OK || cat pipout

In [ ]:
%matplotlib inline
from cognite.seismic import CogniteSeismicClient
import geojson
from getpass import getpass
from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles
import json
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import pandas
from shapely import wkt

client = CogniteSeismicClient(base_url="api.cognitedata.com", api_key=getpass("Enter your Cognite API key: "))




### List available surveys

We can list available surveys, and obtain more detailed information about individual surveys.

In [ ]:
surveys_response = client.survey.list()
print(f"Retrieved {len(surveys_response.surveys)} surveys.")
print(surveys_response)
#print(f"Last survey: \n{surveys_response.surveys[-1]}")


### Show a survey's metadata

Surveys include some metadata. The set of data currently included in the MVP project is limited but can be expanded.

In [ ]:
#Select a survey from the list of surveys 
survey_name="BM-C-4.NSA.11125"

surveydata = client.survey.get(name=survey_name, include_metadata=True)
pandas.DataFrame.from_dict(surveydata.survey.metadata, orient='index', columns=[surveydata.survey.name])

### Survey search
We can also search for surveys with a particular metadata value - in this case we can search for surveys from Brazil:

In [ ]:
surveysearch = client.survey.search(survey_metadata_filter={"Country": "BRAZIL"}, crs="")
pandas.DataFrame.from_dict([s.metadata for s in surveysearch.surveys])

### List files in a survey

Each survey contains seismic products as files, let's have a look at the selected survey:

In [ ]:
client.survey.get(name=survey_name, list_files=True)

### Header printout

Easy access to headers (text and binary) from the API

In [ ]:
#Select a file from the list of files 
file_id='e33563cc-efee-4dda-92bb-d24b86110528'

raw = client.file.get_text_header(id=file_id, include_raw_header=True).meta.raw_header
for i in range(0, len(raw), 80):
    print(str(raw[i:i+80]))
    
binheader = client.file.get_binary_header(id=file_id)
binheader

### Seismic slice

Seismic slices defined by an inline can be plotted. Defining slice by coordinates will be possible too. 

In [ ]:
#Print seimic file's Inline/Crossline range 
client.file.get_line_range(id=file_id)

In [ ]:
#Select an inline from the range and plot the slice
lineslice = client.slice.get_inline(file_id=file_id, inline=8700)
itraces = lineslice.to_array()
plt.figure(figsize=(20,20))
plt.imshow(itraces.transpose(), cmap='seismic', norm=matplotlib.colors.Normalize(vmin=-4000, vmax=4000))

### Coverage Polygon
We have access to a polygon describing the spatial extent of the seismic data:

In [ ]:
edm50 = "EPSG:23031"

coverage = client.file.get_file_coverage(id=file_id, in_wkt=True)
polygon = wkt.loads(coverage.polygon.wkt.geometry)

plt.figure(figsize=(10,8))
x, y = polygon.exterior.xy
plt.plot(x, y)




In [ ]:
#Print coverage polygon coordinatets
polygon.wkt

### Define and download a cut-out SEG-Y by coordinates

Plot coverage polygon once again to define a cut-out within the polygon using coordinates

In [ ]:
#Plot the survey coverage polygon
coverage = client.file.get_file_coverage(id=file_id, in_wkt=True)
polygon = wkt.loads(coverage.polygon.wkt.geometry)
plt.figure(figsize=(10,8))
x, y = polygon.exterior.xy
plt.plot(x, y)

# Define Cut-out by coordinates (useful to print coverage polygon coordinates and follow same schema)
square = "POLYGON((-4210000 -3130000, -4210000 -3140000, -4200000 -3140000, -4200000 -3130000, -4210000 -3130000))"
minipoly = wkt.loads(square)
x,y = minipoly.exterior.xy
plt.plot(x, y)


Download of cut-out specified above

In [ ]:
with open("output1.segy", "wb") as outfile:
    for piece in client.file.get_segy_by_geometry(id=file_id, crs=edm50, wkt=square):
        outfile.write(bytes(piece.content))

! ls -l output1.segy

### Define and download a cut-out SEG-Y by Inline/Crossline

Small cube download with header print out, based on specific inline and crossline range

In [ ]:
#Print Inline/Xline range
client.file.get_line_range(id=file_id)

In [ ]:
with open("output2.segy", "wb") as outfile:
    for piece in client.file.get_segy_by_lines(id=file_id, top_left_inline=8500, top_left_crossline=10500, bottom_right_inline=8505, bottom_right_crossline=10505):
        outfile.write(bytes(piece.content))
        
! dd if=output2.segy conv=ascii bs=80 count=40 2>/dev/null| sed -e "s/.\{80\}/&\n/g" # this is to convert it to ascii, and make it readable